Apuração da Premiação Tina (Individual)



In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# TODO MUDAR QUANDO AS LOJAS TUMELERO DA 2a ONDA ENTRAREM
# Ondas Telhanorte e Tumelero cujas lojas serão consideradas para apuração
safra_tume=1 # Onda cujas lojas Tumelero serão consideradas
safra_telha=3 #  Onda cujas lojas Telhanorte serão consideradas

In [255]:
# Dia da apuração
hoje = pd.to_datetime('today').date()
# hoje = pd.to_datetime('2022-05-28')
ANO=str(hoje.year)

In [256]:
# Dictionary para extrair empresa
ID_UNN=1

dict_EMPRESAS={1:"TELHANORTE", 4:"TUMELERO"}
empresa=dict_EMPRESAS[ID_UNN]
print(ID_UNN)
print(empresa)

1
TELHANORTE


In [257]:
# DIA PADRÃO DE APURAÇÃO
DIA_APURA=hoje.day

In [258]:
DIA_APURA

28

In [259]:
MESES= ['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
I_MESES= ['01','02','03','04','05','06','07','08','09','10','11','12']
MES=MESES[hoje.month-1]
I_MES=I_MESES[MESES.index(MES)]
I_MES_ANTERIOR=I_MESES[MESES.index(MES)-1]


In [260]:
I_MES_ANTERIOR

'04'

In [261]:
# Período de Apuração   -   Faltas injustificadas no período
# 27 de Janeiro de 2022
DATA_I = pd.to_datetime('2022-'+I_MES_ANTERIOR+'-'+str(DIA_APURA))  # data inicial AAAA-MM-DD
DATA_F = pd.to_datetime('2022-'+I_MES+'-'+str(DIA_APURA-1))  # data final AAAA-MM-DD

In [262]:
print(DATA_I)
print(DATA_F)

2022-04-28 00:00:00
2022-05-27 00:00:00


In [263]:
# PASTAS 

DIR_BASE='C:/apuracao/'
DIR_INFO=DIR_BASE+'info/'
DIR_TINA_BASES=DIR_INFO
DIR_ENTRA=DIR_BASE+'ENTRADA/'
# PASTA PARA MANTER AS BASES ORIGINAIS
DIR_IN=DIR_BASE+ANO+'/'+MES+'/ORIGINAIS/'
# PASTA COM PLANILHA DE CHECAGEM PARA O ANALISTA DE NEGÓCIOS
DIRETORIO_CHECAGEM=DIR_BASE+ANO+'/'+MES+'/CHECAGEM/'
# PASTA PARA PLANILHAS DAS VERBAS PARA RH 
DIR_RH=DIR_BASE+ANO+"/"+MES+"/"+"para_RH/"



In [264]:
# CRIANDO PASTAS DE SAÍDA SE NÃO EXISTIREM
Path(DIR_IN).mkdir(parents=True, exist_ok=True)
Path(DIRETORIO_CHECAGEM).mkdir(parents=True, exist_ok=True)
Path(DIR_RH).mkdir(parents=True, exist_ok=True)


In [265]:
# Arquivos a serem importados
# Incluídas lojas TUMELERO
F_ALVO = DIR_INFO+"target_salesperson.csv" # 
F_VERBAS = DIR_INFO+"/verbas_descricao.xlsx"

F_LOJA_SAFRA=DIR_INFO+"lojas.csv" # 
#f_loja_safra = "info/lojas_pilotos_safra3.xlsx" # Exemplo de janeiro


In [266]:
# TODO Colocar os valores dos parâmetros em arquivo e importar 
# Parâmetros do Modelo
# Modelo Vendedores
# Modelo Vendedores Comissão
gat_min_comissao = 0.0  # Gatilho minimo comissão
ANG_COEF_COMISSAO = 0.01  # Coeficiente angular = 1%
lin_coef_comissao = 0.0  # Coeficiente Linear = 0.0%

# Modelo Vendedores Comissão Turbinada Decoração
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
gat_min_decoracao = 0.0  # Gatilho minimo decoração
ANG_COEF_DECORACAO = 0.02  # Coeficiente angular decoração = 2%
lin_coef_decoracao = 0.0  # Coeficiente Linear decoração = 0.0%

# Modelo Vendedores Comissão turbinada Áreas Técnicas
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
gat_min_areastecnicas = 0.0  # Gatilho minimo Áreas Técnicas = 0.0%
ANG_COEF_AREASTECNICAS= 0.005  # Coeficiente angular Áreas Técnicas = 0.5%
lin_coef_areastecnicas = 0.0  # Coeficiente Linear decoração = 0.0%

# Modelo Vendedores Prêmio superação vendas
M1 = 100  # meta1 = 100%
M2 = 150  # meta2 = 150%
M3 = 200  # meta3 = 200%
M4 = 250  # meta4 = 250%
B1 = {1:300, 4:200}  # bonus1 -> ganha se bater a meta1; Telhanorte 1 = R$300,00; Tumelero 4= R$200,00
B2 = {1:600, 4:400}  # bonus2 -> ganha se bater a meta2; Telhanorte 1 = R$600,00; Tumelero 4= R$400,00
B3 = {1:900, 4:600}  # bonus3 -> ganha se bater a meta3; Telhanorte 1 = R$900,00; Tumelero 4= R$600,00
B4 = {1:1200, 4:800}  # bonus4  -> ganha se bater a meta4; Telhanorte 1 = R$1200,00; Tumelero 4= R$800,00

# Modelo Vendedores Prêmio Rentabilidade
GAT_MIN_RENTAB = 100.  # Tem que bater 100% da rentabilidade
ANG_COEF_RENTAB = 0.001  # Coeficiente angular da rentabilidade = 0.1%
LIN_COEF_RENTAB = -0.1  # Coeficiente linear da rentabilidade = -0.1
KICKER = {1:200., 4:100} # Premiação inicial quando a rentabilidade alcança 100%; 
                                   # Telhanorte 1 = R$200,00; Tumelero 4 = R$100,00

In [267]:
# Parâmetros do Modelo
# Modelo Assessores
ASS_A1 = 30000  # Alvo1 = R$30.000,00
ASS_A2 = 50000  # Alvo2 = R$50.000,00
ASS_B1 = 100  # Bônus1 caso bata Alvo1
ASS_B2 = 200  # Bônus2 caso bata Alvo2


In [268]:
# Vitor (Wingg) via e-mail no dia da apuração
E_EXTRATO = DIR_ENTRA+"extract_remuneration_"+ANO+I_MES+str(DIA_APURA)+".xlsx"
F_EXTRATO = DIR_IN+"extract_remuneration_"+ANO+I_MES+str(DIA_APURA)+".xlsx"

# Thayna via sharepoint antes da apuração
# F_FALTAS = DIR_IN+"FALTAS_INJUSTIFICADAS_PREMIACOES TINA_23mar2022_ThaynaCarneiro.xlsx"
E_FALTAS = DIR_ENTRA+"PREMIACOES TINA.xlsx"
F_FALTAS = DIR_IN+"PREMIACOES TINA.xlsx"

In [269]:
print('A planilha de checagem da Apuração da PREMIAÇÃO TINA INDIVIDUAL realizada em '+MES+' está aqui:')
print(DIRETORIO_CHECAGEM)
# ARQUIO DE SAÍDA PARA CHECAGEM DO ANALISTA DE NEGÓCIOS - EM EXCEL
F_SAI_CHECAGEM=DIRETORIO_CHECAGEM + "CHECAGEM_APURACAO_PREMIACAO_INDIVIDUAL_"+MES+".xlsx"
print(F_SAI_CHECAGEM)


A planilha de checagem da Apuração da PREMIAÇÃO TINA INDIVIDUAL realizada em mai está aqui:
C:/apuracao/2022/mai/CHECAGEM/
C:/apuracao/2022/mai/CHECAGEM/checagem_apuração_PREMIACAO_INDIVIDUAL_mai.xlsx


In [270]:
# Sufixos possíveis
sufixo="_sem_considerar_faltas_injustificadas"
#sufixo="_analista"

In [271]:
def nome_verbas(DIR_RH, empresa):
    # Verbas para o RH das premiações Tina Individuais
    F_VERBA_ASS=DIR_RH+"VERBA_4118_SEGMENTO_ASSESSORES_"+empresa+".xlsx"
    F_VERBA_VEN_REN=DIR_RH+"VERBA_4119_RENTABILIDADE_VENDAS_"+empresa+".xlsx"
    F_VERBA_VEN_DEC=DIR_RH+"VERBA_4120_DECORACAO_VENDAS_"+empresa+".xlsx"
    F_VERBA_VEN_SUP=DIR_RH+"VERBA_4121_SUPERACAO_VENDAS_"+empresa+".xlsx"
    F_VERBA_VEN_TEC=DIR_RH+"VERBA_4122_AREAS_TECNICAS_VENDAS_"+empresa+".xlsx"
    # Verbas para o RH das premiações Tina Individuais SEM CONSIDERAR FALTAS INJUSTIFICADAS
    F_VERBA_ASS_SEM=DIR_RH+"VERBA_4118_SEGMENTO_ASSESSORES_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_REN_SEM=DIR_RH+"VERBA_4119_RENTABILIDADE_VENDAS_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_DEC_SEM=DIR_RH+"VERBA_4120_DECORACAO_VENDAS_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_SUP_SEM=DIR_RH+"VERBA_4121_SUPERACAO_VENDAS_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_TEC_SEM=DIR_RH+"VERBA_4122_AREAS_TECNICAS_VENDAS_"+empresa+sufixo+".xlsx"

    return {'F_VERBA_ASS':F_VERBA_ASS, 
        'F_VERBA_VEN_REN':F_VERBA_VEN_REN, 
        'F_VERBA_VEN_DEC':F_VERBA_VEN_DEC, 
        'F_VERBA_VEN_SUP':F_VERBA_VEN_SUP, 
        'F_VERBA_VEN_TEC':F_VERBA_VEN_TEC,
        'F_VERBA_ASS_SEM':F_VERBA_ASS_SEM,
        'F_VERBA_VEN_REN_SEM':F_VERBA_VEN_REN_SEM, 
        'F_VERBA_VEN_DEC_SEM':F_VERBA_VEN_DEC_SEM, 
        'F_VERBA_VEN_SUP_SEM':F_VERBA_VEN_SUP_SEM, 
        'F_VERBA_VEN_TEC_SEM':F_VERBA_VEN_TEC_SEM}

In [272]:
# Nomes dos arquivos de verbas para RH
verbas_telha=nome_verbas(DIR_RH,dict_EMPRESAS[1])
verbas_tume=nome_verbas(DIR_RH,dict_EMPRESAS[4])

In [273]:
verbas_telha['F_VERBA_ASS_SEM']

'C:/apuracao/2022/mai/para_RH/VERBA_4118_SEGMENTO_ASSESSORES_TELHANORTE_sem_considerar_faltas_injustificadas.xlsx'

In [274]:
import shutil
# Fazendo backup das bases nas pasta ORIGINAIS respectiva ao mes e ano da apuração 
shutil.copy2(E_EXTRATO,F_EXTRATO)
shutil.copy2(E_FALTAS,F_FALTAS)

'C:/apuracao/2022/mai/ORIGINAIS/PREMIACOES TINA.xlsx'

In [275]:
# Carregando lojas TINA da Telhanorte e da Tumelero 
#
nome_lojas = pd.read_csv(F_LOJA_SAFRA, encoding="ISO-8859-1", delimiter=";")
# Deletando lojas que não estão na safra TINA
nome_lojas.drop(nome_lojas[(
    (nome_lojas['id_unn']==1) & (nome_lojas['safra']>safra_telha)
    ) | (
    (nome_lojas['id_unn']==4) & (nome_lojas['safra']>safra_tume)   
    )
    ].index, inplace=True)

In [276]:
# Função para importar comissões separamente por empresa (vendas da ADP)
def importa_comissoes(arquivo):
    # Atribuindo nomes às colunas das comissões
    column_names=[
    'N1',	
    'N2',	
    'N3',	
    'MATRICULA',	
    'POS_NEG',	 
    'VALOR',
    'NONE'
    ]
    print(arquivo)
    print("Importando as comissões")
    print("Arquivo: "+arquivo)
    comissao = pd.read_csv(arquivo, delimiter=";", names=column_names, header=None, skiprows=1)
    # Trocando "," por "." para poder calcular os números no Python
    comissao["VALOR"]=comissao["VALOR"].str.replace(",",".").astype(float)
    ##  Vendas devolvidas como negativas
    print("Valores negativos para vendas devolvidas")
    comissao["VALOR_COR"]=np.where(comissao["POS_NEG"]==3771, -comissao["VALOR"],comissao["VALOR"])
    # Só matrícula e valor das vendas correspondente a essa matrícula
    comissao=comissao[['MATRICULA','VALOR_COR']]
    # Somando as vendas totais de cada matrícula
    n_vendas = comissao.groupby(['MATRICULA']).size().reset_index()
    n_vendas.rename(columns={0:'NUMERO_DE_LOJAS_COM_VENDA'}, inplace=True)
    print("Somando todas as vendas de uma matrícula:")
    comissao = comissao.groupby(['MATRICULA'])['VALOR_COR'].sum().reset_index()
    comissao.rename(columns={'VALOR_COR':'VENDAS_TOT'}, inplace=True)
    comissao['COMISSAO']=comissao['VENDAS_TOT']*0.01
    comissao=pd.merge(comissao,n_vendas, how='left', on=['MATRICULA'] )
    return comissao

In [277]:
# Comissão TELHANORTE
# Ricardo Valdir via e-mail para o Junior no dia da apuração
E_COMISSAO = DIR_ENTRA+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[1].lower() # base de entrada Comissao
F_COMISSAO = DIR_IN+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[1].lower() # cópia da base Comissão a ser mantida
# backup da base de comissão TELHANORTE
shutil.copy2(E_COMISSAO,F_COMISSAO)
# importando a comissão Telhanorte
comissao_telha=importa_comissoes(F_COMISSAO)


C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_telhanorte
Importando as comissões
Arquivo: C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_telhanorte
Valores negativos para vendas devolvidas
Somando todas as vendas de uma matrícula:


In [278]:
# Para visualizar as vendas totais e a comissão de um colaborador 
comissao_telha[comissao_telha['MATRICULA'] == 25724]

,MATRICULA,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA
435,25724,79905.1,799.051,3


In [279]:
# Comissão TUMELERO
# Ricardo Valdir via e-mail para o Junior no dia da apuração
E_COMISSAO = DIR_ENTRA+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[4].lower() # base de entrada Comissao
F_COMISSAO = DIR_IN+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[4].lower() # cópia da base Comissão a ser mantida
# backup da base de comissão TUMELERO
shutil.copy2(E_COMISSAO,F_COMISSAO)
# importando a comissão Tumelero
comissao_tume=importa_comissoes(F_COMISSAO)


C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_tumelero
Importando as comissões
Arquivo: C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_tumelero
Valores negativos para vendas devolvidas
Somando todas as vendas de uma matrícula:


In [281]:
# juntando comissão da Telhanorte e Tumelero
comissao_telha['ID_UNN']=1 # Identidade da empresa Telhanorte = 1
comissao_tume['ID_UNN']=4 # Identidade da empresa Tumelero = 4
comissao=pd.concat([
    comissao_telha[['ID_UNN', 'MATRICULA', 'VENDAS_TOT', 'COMISSAO', 'NUMERO_DE_LOJAS_COM_VENDA']],
    comissao_tume[['ID_UNN', 'MATRICULA', 'VENDAS_TOT', 'COMISSAO', 'NUMERO_DE_LOJAS_COM_VENDA']]
    ])



In [282]:
# Lendo os alvos que os vendedores devem atingir por loja
print("Lendo alvos das lojas no arquivo:")
print(F_ALVO)
alvos = pd.read_csv(F_ALVO, delimiter=";", header=0, skiprows=0)
# Renomeando e transformando em número
alvos.rename(columns={'target_sales_salesperson':'Alvo1'}, inplace=True)
alvos['Alvo1']=alvos['Alvo1'].str.replace(",",".").astype(float)
#Transformando floor remuneration em Mínimo Garantido e em número
alvos['Minimo Garantido']=alvos['floor_remuneration'].str.replace(",",".").astype(float)

Lendo alvos das lojas no arquivo:
C:/apuracao/info/target_salesperson.csv


In [284]:
# Mantendo somente alvos Telhanorte
#alvos=alvos[alvos['id_unn']==1] # id_unn = (1=Telhanorte; 2=Tumelero)
# Mantendo somente alguns parâmetros
alvos= alvos[['cod_loja','Alvo1', 'Minimo Garantido']]

alvos=pd.merge(alvos,nome_lojas[['cod_loja','sigla_loja']],how='left',on='cod_loja')

In [288]:
# Lendo faltas injustificadas
print("Lendo faltas injustificadas no arquivo:")
print(F_FALTAS)
faltas_inj = pd.read_excel(F_FALTAS, header=0)

Lendo faltas injustificadas no arquivo:
C:/apuracao/2022/mai/ORIGINAIS/PREMIACOES TINA.xlsx


In [289]:
faltas = faltas_inj[(faltas_inj['DATA DA OCORRÊNCIA'] >= DATA_I) &
           (faltas_inj['DATA DA OCORRÊNCIA'] <= DATA_F)]
faltas['DATA DA OCORRÊNCIA'].sort_values()

10    2022-04-28
83    2022-04-28
121   2022-04-28
133   2022-04-28
36    2022-04-28
         ...    
307   2022-05-26
329   2022-05-27
308   2022-05-27
340   2022-05-27
322   2022-05-27
Name: DATA DA OCORRÊNCIA, Length: 225, dtype: datetime64[ns]

In [290]:
faltas=pd.merge(
    faltas.rename(columns={
        'EMPRESA':'ID_UNN',
        'NOME ESTAB':'SIGLA_LOJA'
        }),
    nome_lojas[['id_unn','cod_loja','sigla_loja']].rename(columns={
        'id_unn':'ID_UNN',
        'cod_loja':'COD_LOJA',
        'sigla_loja': 'SIGLA_LOJA'
        }),
    how='left',
    on=['ID_UNN', 'SIGLA_LOJA']
    )

In [292]:
# Loading Remuneration extract from Tina
print("Lendo Extrato de Remuneração da Tina")
print(F_EXTRATO)
extrato = pd.read_excel(F_EXTRATO)

Lendo Extrato de Remuneração da Tina
C:/apuracao/2022/mai/ORIGINAIS/extract_remuneration_20220528.xlsx


In [293]:
# Somente colaboradores Tina das lojas da safra
print("Considerando somente colaboradores das lojas participantes da Tina")
COLS_TO_KEEP=[
    'store_sk', 
    'registration_id', 
    'segment', 
    'job_simplified',
    'E1_gross_sales', 
    'E2_gross_sales', 
    'E3_gross_sales', 
    'gross_sales', 
    'sales_segment', 
    'rentability'
    ]
so_safra = extrato[extrato["store_sk"].isin(nome_lojas['cod_loja'])][COLS_TO_KEEP].copy(deep=True)
so_safra.columns

Considerando somente colaboradores das lojas participantes da Tina


Index(['store_sk', 'registration_id', 'segment', 'job_simplified',
       'E1_gross_sales', 'E2_gross_sales', 'E3_gross_sales', 'gross_sales',
       'sales_segment', 'rentability'],
      dtype='object')

In [295]:
#para planilha checagem
tina=so_safra.copy(deep=True)


In [296]:
# Verificando quem tem falta injustificada
tina['FALTA']=np.where(
    (tina['store_sk'].isin(faltas['COD_LOJA'])==True) &
    (tina['registration_id'].isin(faltas['MATR EMPR'])==True) 
    ,'SIM','NAO')

In [338]:
faltas['COD_LOJA']

0          34.0
1          34.0
2          34.0
3          34.0
4          34.0
         ...   
220    510078.0
221    510078.0
222    510078.0
223    510078.0
224         NaN
Name: COD_LOJA, Length: 225, dtype: float64

In [298]:
# Juntando nome de sigla das lojas
tina=pd.merge(
    tina.rename(columns={'store_sk':'cod_loja'}),
    nome_lojas[['cod_loja', 'sigla_loja']],
    how='left',on='cod_loja'
    )

In [299]:
# Renomeando matricula para juntar com a comissão
tina.rename(columns={'registration_id':'MATRICULA'}, inplace=True)


In [300]:
# Identificando lojas da empresa (ID_UNN) Telhanorte = 1 e Tumelero = 4
tina['ID_UNN']=np.where(tina['cod_loja']<510000,1,4)

In [301]:
# # Juntando com a comissão
tina=tina.merge(
    comissao,
    how='left',on=['ID_UNN','MATRICULA'])

In [303]:

tina=pd.merge(
    tina,
    alvos[['cod_loja','Alvo1', 'Minimo Garantido']],
    how='left', on='cod_loja'
    )

In [305]:
# Eliminando alvos de assessores
#VENDAS_SOBRE_ALVO da TUMELERO: baseado em vendas totais da Tina que considera devoluções (Vendas ADP não considera devoluções)
# VENDAS_SOBRE_ALVO da TELHANORTE: baseado em vendas da ADP considera devoluções
tina.loc[tina.job_simplified=='assistant', 'Alvo1'] = None
tina['VENDAS_SOBRE_ALVO'] =  None
tina.loc[(tina.ID_UNN == 1) & (tina.job_simplified=='salesperson'), 'VENDAS_SOBRE_ALVO'] = tina['VENDAS_TOT']/tina['Alvo1']
tina.loc[(tina.ID_UNN == 4) & (tina.job_simplified=='salesperson'), 'VENDAS_SOBRE_ALVO'] = tina['gross_sales']/tina['Alvo1']


,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,ID_UNN,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido,VENDAS_SOBRE_ALVO
621,39,27960,E1,assistant,9291.757,6945.977,15428.828,31666.562,9291.757,-570.350524,NAO,UBE,1,31666.56,316.6656,1.0,NaN,1400.4,None
527,32,24922,NaN,salesperson,6822.458,757.158,27243.045,34822.661,NaN,102.386038,NAO,GRU,1,34822.66,348.2266,1.0,245148.00,2042.9,0.142047
169,9,29630,E1,assistant,32419.161,18987.432,45917.512,97324.105,32419.161,-2.193520,NAO,SA1,1,97324.11,973.2411,2.0,NaN,1983.7,None
627,39,29231,E1,assistant,9233.455,3799.059,16518.565,29551.079,9233.455,106.259212,NAO,UBE,1,29551.08,295.5108,1.0,NaN,1400.4,None
603,38,26059,E3,salesperson,17169.839,11463.312,195355.149,223988.300,195355.149,104.170546,NAO,DPE,1,223988.30,2239.8830,1.0,198000.00,1929.2,1.131254
669,43,27647,E1,salesperson,43841.091,13534.721,60680.798,118056.610,43841.091,-465.471395,NAO,TAT,1,118056.61,1180.5661,1.0,238200.00,1985.0,0.49562
264,23,23976,E3,salesperson,1016.430,NaN,2337.640,3354.070,2337.640,104.961797,NAO,PMG,1,3354.07,33.5407,1.0,238200.00,1985.0,0.014081
623,39,28225,E3,salesperson,15212.523,8662.295,131130.772,155005.590,131130.772,-143.765657,NAO,UBE,1,155005.59,1550.0559,1.0,160000.00,1400.4,0.968785
894,75,101,NaN,-,-383.460,-356.900,-363.300,-1103.660,NaN,106.746306,NAO,AMZ,1,-1103.66,-11.0366,1.0,160000.00,1283.7,None
788,64,20948,E1,salesperson,40956.348,19346.085,271583.585,331886.018,40956.348,-26280.364072,NAO,SAN,1,331886.02,3318.8602,1.0,256003.20,2133.4,1.296414


In [307]:
# Eliminando minimos garantidos de assessores
tina['Minimo Garantido']=np.where(tina['job_simplified']=='assistant',np.nan,tina['Minimo Garantido'])

In [308]:
# Vendedores atingiu alvo ou não atingiu alvo
tina.loc[
    (tina.job_simplified=='salesperson') &
    (tina.VENDAS_SOBRE_ALVO>=1.), 'ATINGIU_ALVO'] = 'SIM'
tina.loc[
    (tina.job_simplified=='salesperson') &
    (tina.VENDAS_SOBRE_ALVO<1.), 'ATINGIU_ALVO'] = 'NAO'

# TODO DELETAR LINHA SE 
# tina['ATINGIU_ALVO']=np.where(tina['VENDAS_SOBRE_ALVO']>=1.,'SIM','NAO')

In [309]:
# Assessores atingiu alvo
tina.loc[
    (tina.job_simplified=='assistant') &
    (tina.sales_segment>=30000), 'ATINGIU_ALVO'] = '30mil'
tina.loc[
    (tina.job_simplified=='assistant') &
    (tina.sales_segment>=50000), 'ATINGIU_ALVO'] = '50mil'
tina.loc[
    (tina.job_simplified=='assistant') &
    (tina.sales_segment<30000), 'ATINGIU_ALVO'] = 'NAO'


In [310]:
# RECEBE PREMIAÇÃO TINA SE ATINGIU ALVO E NÃO TEM FALTA INJUSTIFICADA
tina['RECEBE_PREMIACAO']=np.where((
    (tina['ATINGIU_ALVO']=='SIM') |
    (tina['ATINGIU_ALVO']=='50mil') |
    (tina['ATINGIU_ALVO']=='30mil')) & 
    (tina['FALTA']=='NAO'),
     'SIM', 'NAO')


In [311]:
tina[['ID_UNN', 'MATRICULA', 'gross_sales','VENDAS_TOT', 'Alvo1', 'ATINGIU_ALVO','RECEBE_PREMIACAO']][tina['RECEBE_PREMIACAO']!='NAO']
#tina[['MATRICULA','VENDAS_TOT', 'FALTA', 'ATINGIU_ALVO','RECEBE_PREMIACAO']][tina['ATINGIU_ALVO']!='NAO'].sample(15)
#tina[['MATRICULA','VENDAS_TOT', 'FALTA', 'ATINGIU_ALVO','RECEBE_PREMIACAO']][tina['FALTA']=='SIM'].sample(15)

,ID_UNN,MATRICULA,gross_sales,VENDAS_TOT,Alvo1,ATINGIU_ALVO,RECEBE_PREMIACAO
0,1,1890,418604.782,418604.78,268704.0,SIM,SIM
4,1,10522,530005.184,530005.18,268704.0,SIM,SIM
5,1,13170,381787.635,381787.63,268704.0,SIM,SIM
12,1,27242,92077.611,92077.61,NaN,50mil,SIM
13,1,27359,100169.107,97310.11,NaN,30mil,SIM
...,...,...,...,...,...,...,...
978,4,7895,187026.370,188281.88,173500.0,SIM,SIM
983,4,11412,190274.700,195523.31,180000.0,SIM,SIM
1008,4,10151,173593.780,178678.13,173500.0,SIM,SIM
1011,4,11949,186216.210,202854.24,173500.0,SIM,SIM


In [312]:
# FUNÇÃO para salvar verba para TELHANORTE ou TUMELERO
# Entradas: v verba, n nome_verba, x numero_verba, f f_nome
def salva_verba(empresa,verba,numero_verba,arquivo):
#    v.rename(columns={ v.columns[0]: 'MATRICULA'}, inplace=True)
    verba.rename(columns={ verba.columns[1]: "VALOR" }, inplace=True)
    verba['ZERO'] = 0
    verba['VERBA'] = numero_verba
    if empresa == 'TELHANORTE':
        verba=verba[['MATRICULA', 'ZERO', 'VERBA','VALOR']]
    elif empresa == 'TUMELERO':
         verba=verba[['MATRICULA', 'VERBA', 'ZERO','VALOR']]
    else: 
        print('Erro!!! Não veio a empresa nem como TELHANORTE e nem como TUMELERO')
        return
    verba = verba[verba['VALOR'] > 0]
    # Saving as a excel file for RH
    verba.to_excel(arquivo, sheet_name="VERBA "+str(numero_verba), index=False)
    

In [316]:
def alvo_bonus_rentabilidade(id_unn, vendas_tot,rentabilidade,vendas_seg,alvo):
#     # Modelo Vendedores Prêmio Rentabilidade
#    T E S T A D O     E     C H E C A D O
# gat_min_rentab = 100.  # Tem que bater 100% da rentabilidade
# ang_coef_rentab = 0.001  # Coeficiente angular da rentabilidade = 0.1%
# lin_coef_rentab = -0.1  # Coeficiente linear da rentabilidade = -0.1
# kicker = 200 TELHANORTE            kicker = 100 TUMELERO
    global GAT_MIN_RENTAB, ANG_COEF_RENTAB, LIN_COEF_RENTAB, KICKER
    if vendas_tot>=alvo and rentabilidade >= GAT_MIN_RENTAB:
         val_rent=(rentabilidade*ANG_COEF_RENTAB+LIN_COEF_RENTAB)*vendas_seg+KICKER[id_unn]
    else:
        val_rent=0.0
    return val_rent



In [317]:
# Calculando Prêmio Rentabilidade para TELHANORTE
# Vendas Tumelero: Vendas totais vindo da Tina (desconta devoluções) pois VENDAS_ADP da Tumelero não desconta devoluções
tina.loc[
    (tina.ID_UNN == 1) &  # ID_UNN para Telhanorte
    (tina.job_simplified=='salesperson'), 
    'PREMIO_RENTABILIDADE'
    ] = tina.apply(
        lambda x: alvo_bonus_rentabilidade(
            1,  # ID_UNN para Telhanorte
            x['VENDAS_TOT'], x['rentability'], x['sales_segment'], x['Alvo1']# Vendas Telha: Vendas totais Tina (desconta devoluções) pois VENDAS_ADP da Tumelero não desconta devoluções
            ), axis=1).copy(deep=True)

# Calculando Prêmio Rentabilidade para TUMELERO
tina.loc[
    (tina.ID_UNN == 4) &  # ID_UNN para Tumelero
    (tina.job_simplified=='salesperson'), 
    'PREMIO_RENTABILIDADE'
    ] = tina.apply(
        lambda x: alvo_bonus_rentabilidade(
            4, # ID_UNN para Tumelero
            x['gross_sales'], x['rentability'], x['sales_segment'], x['Alvo1']# Vendas Tume: Vendas totais Tina (desconta devoluções) pois VENDAS_ADP da Tumelero não desconta devoluções
            ), axis=1).copy(deep=True)

In [318]:
# Salva verba rentabilidade TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], # ID_UNN TELHA
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][tina['ID_UNN']==1].copy(deep=True),  # ID_UNN TELHA
    4119,
    verbas_telha['F_VERBA_VEN_REN_SEM']) # verbas_telha['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[1],  # ID_UNN TELHA
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][
        (tina['ID_UNN']==1) &  # ID_UNN TELHA
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4119,
        verbas_telha['F_VERBA_VEN_REN'])# verbas_telha

# Salva verba rentabilidade TUMELERO -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[4], # ID_UNN TUMELERO
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][tina['ID_UNN']==4].copy(deep=True),# ID_UNN TUMELERO
    4119,
    verbas_tume['F_VERBA_VEN_REN_SEM']) # verbas_tume sem considerar faltas injustificadas
salva_verba(
    dict_EMPRESAS[4],  # ID_UNN TUMELERO
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][
        (tina['ID_UNN']==4) &  # ID_UNN TUMELERO
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4119,
        verbas_tume['F_VERBA_VEN_REN']) # verbas_tume['F_VERBA_VEN_REN']


In [319]:
# Calculating bonus for SALESPERSON TELHANORTE E TUMELERO
# NÃO TEM DIFERENÇA ENTRE EMPRESAS
# Verba 4120 - Segmento vendedores - DECORAÇÃO
def alvo_bonus_decoracao(vendas_seg,vendas_tot,alvo):
# Modelo Vendedores Prêmio DECORAÇÃO vendas
#    T E S T A D O     E     C H E C A D O
# Modelo Vendedores Comissão Turbinada Decoração
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
# gat_min_decoracao = 0.0  # Gatilho minimo decoração
# ang_coef_decoracao = 0.02  # Coeficiente angular decoração = 2%
# lin_coef_decoracao = 0.0  # Coeficiente Linear decoração = 0.0%

    global ANG_COEF_DECORACAO 
    if vendas_tot>=alvo:
        bonus = vendas_seg*ANG_COEF_DECORACAO
    else:
        bonus=0.0
    return bonus


In [320]:
# Calculando Verba DECORAÇÃO para TELHANORTE 
# Vendas Telhanorte: Vendas totais vindo da ADP (desconta devoluções)
tina.loc[
    (tina.ID_UNN==1) &  # ID_UNN TELHA
    (tina.job_simplified=='salesperson'), 
    'PREMIO_DECORACAO'
    ] = tina.apply(
        lambda x: alvo_bonus_decoracao(
            x['E2_gross_sales'], x['VENDAS_TOT'], x['Alvo1'] # vendas totais ADP
            ), axis=1).copy(deep=True)

# Calculando Verba DECORAÇÃO para TUMELERO 
# Vendas Tumelero: Vendas totais vindo da Tina (desconta devoluções) pois VENDAS_ADP da Tumelero não desconta devoluções
tina.loc[
    (tina.ID_UNN==4) &  # ID_UNN TUME
    (tina.job_simplified=='salesperson'), 
    'PREMIO_DECORACAO'
    ] = tina.apply(
        lambda x: alvo_bonus_decoracao(
            x['E2_gross_sales'], x['gross_sales'], x['Alvo1'] # VENDAS totais Tina
            ), axis=1).copy(deep=True)


In [321]:
# Salva verba decoracao TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_DECORACAO']][tina['ID_UNN']==1].copy(deep=True),
    4120,
    verbas_telha['F_VERBA_VEN_DEC_SEM']) # verbas_telha
salva_verba( # sem considerar faltas injustificadas
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_DECORACAO']][
        (tina['ID_UNN']==1) &
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4120,
        verbas_telha['F_VERBA_VEN_DEC']) # verbas_telha

# Salva verba decoracao TUMELERO -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[4], 
    tina[['MATRICULA','PREMIO_DECORACAO']][tina['ID_UNN']==4].copy(deep=True),
    4120,
    verbas_tume['F_VERBA_VEN_DEC_SEM']) # verbas_tume  sem considerar faltas injustificadas
salva_verba(
    dict_EMPRESAS[4], 
    tina[['MATRICULA','PREMIO_DECORACAO']][
        (tina['ID_UNN']==4) &
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4120,
        verbas_tume['F_VERBA_VEN_DEC']) # verbas_tume

In [322]:
# Calculating bonus for SALESPERSON considering differences for TELHANORTE e TUMELERO 
def alvo_bonus_superacao(id_unn, vendas_tot): # id_unn 1=Telhanorte e 4=Tumelero
# Modelo Vendedores Prêmio superação vendas
# T E S T A D O    E    C H E C A D O 
#M1 = 100  # meta1 = 100%
#M2 = 150  # meta2 = 150%
#M3 = 200  # meta3 = 200%
#M4 = 250  # meta4 = 250%
#B1 = {1:300, 4:200}  # bonus1 -> ganha se bater a meta1; Telhanorte 1 = R$300,00; Tumelero 4= R$200,00
#B2 = {1:600, 4:400}  # bonus2 -> ganha se bater a meta2; Telhanorte 1 = R$600,00; Tumelero 4= R$400,00
#B3 = {1:900, 4:600}  # bonus3 -> ganha se bater a meta3; Telhanorte 1 = R$900,00; Tumelero 4= R$600,00
#B4 = {1:1200, 4:800}  # bonus4  -> ganha se bater a meta4; Telhanorte 1 = R$1200,00; Tumelero 4= R$800,00
    global M1, M2, M3, M4, B1, B2, B3, B4 # metas e respectivos bonus
    if vendas_tot>=M4:
        bonus = B4[id_unn]
    elif vendas_tot>=M3:
        bonus = B3[id_unn]
    elif vendas_tot>=M2:
        bonus= B2[id_unn]
    elif vendas_tot>=M1:
        bonus=B1[id_unn]
    else:
        bonus=0.0
    return bonus



In [323]:
# Calculando Prêmio Superação para TELHANORTE
tina.loc[
    (tina.ID_UNN == 1) &  # ID_UNN para TELHANORTE
    (tina.job_simplified=='salesperson'), 
   'PREMIO_SUPERACAO'
    ] = tina.apply(
        lambda x: alvo_bonus_superacao(
            1, # ID_UNN para TELHANORTE
            x['VENDAS_TOT']/x['Alvo1']*100. 
            ), axis=1).copy(deep=True)

# Calculando Prêmio Superação para TUMELERO
tina.loc[
    (tina.ID_UNN == 4) &  # ID_UNN para Tumelero
    (tina.job_simplified=='salesperson'), 
   'PREMIO_SUPERACAO'
    ] = tina.apply(
        lambda x: alvo_bonus_superacao(
            4, # ID_UNN para Tumelero
            x['gross_sales']/x['Alvo1']*100. # Vendas Tumelero considerando devoluções (vendas Tina pois vendas_ADP não considera descontos)
            ), axis=1).copy(deep=True)


In [324]:
# Salva verba superação TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], #
    tina[['MATRICULA','PREMIO_SUPERACAO']][tina['ID_UNN']==1].copy(deep=True),
    4121,
    verbas_telha['F_VERBA_VEN_SUP_SEM']) # verbas_telha['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_SUPERACAO']][
        (tina['ID_UNN']==1) &
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4121,
        verbas_telha['F_VERBA_VEN_SUP'])

# Salva verba superação TUMELERO -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[4], # TUMELERO 
    tina[['MATRICULA','PREMIO_SUPERACAO']][tina['ID_UNN']==4].copy(deep=True),
    4121,
    verbas_tume['F_VERBA_VEN_SUP_SEM']) # verbas TUMELERO['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[4], # TUMELERO 
    tina[['MATRICULA','PREMIO_SUPERACAO']][
        (tina['ID_UNN']==4) & # TUMELERO 
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4121,
        verbas_tume['F_VERBA_VEN_SUP']) # verbas TUMELERO


In [325]:
# para TELHANORTE E TUMELERO SEM DIFERENÇA ENTRE EMPRESAS
def alvo_bonus_areastecnicas(vendas_seg,vendas_tot,alvo):
# Modelo Vendedores Prêmio ÁREAS TÉCNICAS
#    T E S T A D O     E     C H E C A D O
# Modelo Vendedores Comissão  ÁREAS TÉCNICAS
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
# gat_min_areastecnicas = 0.0  # Gatilho minimo Áreas Técnicas = 0.0%
# ang_coef_areastecnicas = 0.005  # Coeficiente angular Áreas Técnicas = 0.5%
# lin_coef_areastecnicas = 0.0  # Coeficiente Linear decoração = 0.0%

    global ANG_COEF_AREASTECNICAS # gat_min_areastecnicas, ang_coef_areastecnicas, lin_coef_areastecnicas 
    if vendas_tot>=alvo:
        bonus = vendas_seg*ANG_COEF_AREASTECNICAS
    else:
        bonus=0.0
    return bonus




In [326]:
# Calculando Verba ATEAS TECNICAS para TELHANORTE E TUMELERO
# DIFERENÇA ENTRE EMPRESAS: 
# Vendas considerando devoluções:

# Telhanorte: Vendas_TOT ADP
tina.loc[
    (tina.ID_UNN==1) &
    (tina.job_simplified=='salesperson'), 
    'PREMIO_AREAS_TECNICAS'
    ] = tina.apply(
        lambda x: alvo_bonus_areastecnicas(
            x['E1_gross_sales'],x['VENDAS_TOT'],x['Alvo1']
            ), axis=1).copy(deep=True)

# Tumelero:   Vendas_Tot Tina (gross sales) pois vendas_TOT ADP não desconta as devoluções
tina.loc[
    (tina.ID_UNN==4) &
    (tina.job_simplified=='salesperson'), 
    'PREMIO_AREAS_TECNICAS'
    ] = tina.apply(
        lambda x: alvo_bonus_areastecnicas(
            x['E1_gross_sales'],x['gross_sales'],x['Alvo1']
            ), axis=1).copy(deep=True)



In [327]:
# Salva verba áreas técnicas TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][tina['ID_UNN']==1].copy(deep=True), # TELHANORTE
    4122,
    verbas_telha['F_VERBA_VEN_TEC_SEM']) # verbas # TELHANORTE
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][
        (tina['ID_UNN']==1) & # TELHANORTE
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4122,
        verbas_telha['F_VERBA_VEN_TEC']) # verbas # TELHANORTE

# Salva verba áreas técnicas TUMELERO -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[4], # TUMELERO=4
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][tina['ID_UNN']==4].copy(deep=True), # TUMELERO=4
    4122,
    verbas_tume['F_VERBA_VEN_TEC_SEM']) # verbas # TUMELERO
salva_verba(
    dict_EMPRESAS[4], # TUMELERO=4
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][
        (tina['ID_UNN']==4) & # TUMELERO=4
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4122,
        verbas_tume['F_VERBA_VEN_TEC']) # verbas TUMELERO


In [328]:
# Calculando Prêmio Assessor SOMENTE TELHANORTE tem assessores
def alvo_bonus_assessor(vendas_seg):
    global ASS_A1, ASS_A2, ASS_B1, ASS_B2 # ass_a1, ass_a2, ass_b1, ass_b2
    if vendas_seg >= ASS_A2:
        bonus = ASS_B2
    elif vendas_seg >= ASS_A1:
        bonus = ASS_B1
    else:
        bonus = 0.0
    return bonus
    

In [329]:
# Calculando prêmio dos assessores - somente TELHANORTE tem assessores
tina['PREMIO_ASSESSOR'] = np.where(
    tina['job_simplified']=='assistant',
    tina.apply(
    lambda x: alvo_bonus_assessor(x['sales_segment']), axis=1).copy(),np.nan
    )

    

In [330]:
# Salva verba assessores somente TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_ASSESSOR']][tina['ID_UNN']==1].copy(deep=True), # TELHANORTE
    4118, #VERBA PREMIAÇÃO ASSESSOR
    verbas_telha['F_VERBA_ASS_SEM']) # verbas # TELHANORTE
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_ASSESSOR']][
        (tina['ID_UNN']==1) & # TELHANORTE
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4118, #VERBA PREMIAÇÃO ASSESSOR
        verbas_telha['F_VERBA_ASS']) # verbas # TELHANORTE


In [331]:
# diferença VendasTOT_ADP - VendasTOT_TINA
tina['DIFERENCA_VENDAS_totADP_totTINA']=tina['VENDAS_TOT']-tina['gross_sales']


In [335]:
# Reorganizando a ordem dos parâmetros
tina = tina[['ID_UNN', 
'cod_loja', 
'sigla_loja', 
'MATRICULA', 
'segment', 
'job_simplified', 
'E1_gross_sales',
'E2_gross_sales', 
'E3_gross_sales', 
'gross_sales', 
'sales_segment',
'rentability', 
'VENDAS_TOT', 
'COMISSAO', 
'NUMERO_DE_LOJAS_COM_VENDA', 
'Alvo1', 
'Minimo Garantido',
'VENDAS_SOBRE_ALVO', 
'ATINGIU_ALVO', 
'FALTA', 
'RECEBE_PREMIACAO',
'PREMIO_RENTABILIDADE', 
'PREMIO_DECORACAO', 
'PREMIO_SUPERACAO',
'PREMIO_AREAS_TECNICAS', 
'PREMIO_ASSESSOR',
'DIFERENCA_VENDAS_totADP_totTINA']]

In [336]:
#Planilha de checagempara o analista de negócios
tina.to_excel(F_SAI_CHECAGEM, index=False)

In [ ]:
# CHECANDO
tina[(tina['ID_UNN']==4) &
#(tina['job_simplified']=='salesperson') &
(tina['RECEBE_PREMIACAO']=='SIM')].sample(13)

#tina[tina['MATRICULA']==3343]